###Kaggle Stuff

In [1]:
!pip install kaggle
!pip install autocorrect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.8/622.8 kB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622364 sha256=1226e601a51904ffea0a5ec032fac56fbc5524f772d0604b5705790f076e39b7
  Stored in directory: /root/.cache/pip/wheels/b5/7b/6d/b76b29ce11ff8e2521c8c7dd0e5bfee4fb1789d76193124343
Successfully built autocorrect


In [2]:
from google.colab import files
files.upload()  # Upload kaggle.json

{}

In [3]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [4]:
! chmod 600 ~/.kaggle/kaggle.json

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [5]:
!kaggle datasets download -d danofer/sarcasm

Dataset URL: https://www.kaggle.com/datasets/danofer/sarcasm
License(s): copyright-authors
 95% 205M/216M [00:01<00:00, 175MB/s]
100% 216M/216M [00:01<00:00, 151MB/s]


In [6]:
!unzip sarcasm.zip

Archive:  sarcasm.zip
  inflating: test-balanced.csv       
  inflating: test-unbalanced.csv     
  inflating: train-balanced-sarc.csv.gz  
  inflating: train-balanced-sarcasm.csv  


###Import Libraries

In [8]:
import pandas as pd
import numpy as np
import spacy
import re
import string
from spacy.lang.en.stop_words import STOP_WORDS as stopwords
from autocorrect import Speller

###Read Data

In [9]:
df = pd.read_csv('train-balanced-sarcasm.csv')
df.head()

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.
3,0,"This meme isn't funny none of the ""new york ni...",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...


###Data Cleaning and Preprocessing

In [10]:
#Remove all na rows
df = df.dropna()
df.isna().sum()

,0
label,0
comment,0
author,0
subreddit,0
score,0
ups,0
downs,0
date,0
created_utc,0
parent_comment,0


In [11]:
#Selecting only the comment and label column
filtered_df = df[df['subreddit'].isin(['news', 'politics', 'worldnews'])]
comments = filtered_df.loc[:,["label", "comment", "parent_comment"]]
comments.head()

,label,comment,parent_comment
0,0,NC and NH.,"Yeah, I get that argument. At this point, I'd ..."
10,0,I think a significant amount would be against ...,I bet if that money was poured into college de...
17,0,because it's what really bothers him... and it...,He actually acts like a moody emo girl on twit...
22,0,Conservatism as an ideology is for sure a reac...,"I still doubt that ""all conservatives stand fo..."
23,0,"Maybe not control, but certainly that is evide...",Today Russian media tweeted out that Wikileaks...


In [12]:
#Word count column
comments["word_count"] = comments["comment"].apply(lambda x: len(x.split()))

comments.sample(5)


,label,comment,parent_comment,word_count
659673,1,And Benghazi; don't forget Benghazi!,New questions raised about U.S.-Saudi relation...,5
389971,1,If the Koreas can do it ...,It might still be impossible though. A wall it...,7
232417,1,HURRAY FOR 2016.,Great Barrier Reef Undergoing a 'Complete Ecos...,3
755573,0,I would start with discretionary spending prog...,We could start with the Census Bureau. With 50...,19
1005935,1,lol,You know I never thought about it that way. Wh...,1


In [13]:
#Capital letter counts
def count_cap(x):
  result = 0
  for c in x:
    if c.isupper():
      result += 1
  return result

comments["capital_count"] = comments["comment"].apply(lambda x: count_cap(x))

comments.sample(5)

,label,comment,parent_comment,word_count,capital_count
597589,1,"Yeah, the DOJ is toothless.",What would make you think courts enforce anyth...,5,4
345302,1,"Worked in DoJ public corruption, can confirm t...","Right, I mean law enforcement guys just *hate ...",22,3
454046,1,Will this hurt his campaign?,The audience frequently seemed on the side of ...,5,1
624511,1,Maybe Buddhist?,I wonder what religion he followed?,2,2
26304,0,"No, if you actually read what they said, it wa...",Then I guess the Portland police were just lyi...,17,1


In [14]:
#Punctuation count
comments["punc_count"] = comments["comment"].apply(lambda x: sum(1 for i in x if i in string.punctuation))
comments.sample(5)

,label,comment,parent_comment,word_count,capital_count,punc_count
992244,0,"As I recall, He wasn't the one drinking his wa...",Fox News Launches a Smear Campaign Accusing Jo...,13,3,3
356790,0,Europeans fucked up the middle east.,"Yes, but who raped who first?",6,1,1
678878,0,Makes sense since the EU is planning on having...,Germany agreed Thursday to turn more than 60 f...,11,3,1
675020,0,Obviously the fluoride did a number on you.,Tell me the one with the pseudo science about ...,8,1,1
851129,1,Racist!,Pot. Kettle. Black.,1,1,1


In [20]:
comments_with_special = comments[comments['comment'].str.contains('@', na=False)]
comments_with_special

,label,comment,parent_comment,word_count,capital_count,punc_count
66,1,I can't wait until @potus starts a twitter war...,Here's what happens when Obama gives up his Tw...,12,3,3
3044,0,"And even better, they'll be coming from @POTUS...",how many headlines these next four years are g...,11,8,5
18342,0,that makes me feel so much better^@#$%,They only really enforce jaywalking when the r...,7,0,5
23967,0,From the assholes @r/the ~~donald~~ dipshit,#StopThePot: Trump trolls launch a new misinfo...,6,1,6
153713,0,What percentage think he is unqualified becaus...,Minnesota Poll: 55 percent say Trump unfit for...,12,1,3
176349,0,@Cragscleftprison Get off the sub if all your ...,Fuck you too,15,3,4
179461,1,Thank you @POTUS for negotiating a nuclear dea...,"Iran Considers Naval Bases in Yemen, Syria",10,7,2
250606,0,Wife is @ $75k after 6 years experience.,I work in a hospital and was curious as to how...,8,1,3
309235,1,Nonsense im with @Boshasaurus_Rex walls never ...,you dont need to patrol a wall as much as open...,13,3,3
314197,0,lmao@ liberals getting screwed by establishmen...,Liberals lash out at Tim Kaine over banking le...,8,1,4


In [ ]:
#Lower case
comments["comment"] = comments["comment"].apply(lambda x: x.lower())
comments["parent_comment"] = comments["parent_comment"].apply(lambda x: x.lower())
comments.head()

,label,comment,parent_comment,word_count,capital_count,punc_count
0,0,nc and nh.,"yeah, i get that argument. at this point, i'd ...",3,4,1
10,0,i think a significant amount would be against ...,i bet if that money was poured into college de...,15,1,1
17,0,because it's what really bothers him... and it...,he actually acts like a moody emo girl on twit...,12,0,6
22,0,conservatism as an ideology is for sure a reac...,"i still doubt that ""all conservatives stand fo...",29,1,2
23,0,"maybe not control, but certainly that is evide...",today russian media tweeted out that wikileaks...,10,1,2


In [ ]:
#List of contractions manually modified for convenience
contractions = {
"i ain't": "i am not",
"you ain't": "you are not",
"he ain't": "he is not",
"she ain't": "she is not",
"they ain't": "they are not",
"it ain't": "it is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": " what is",
"what've": "what have",
"when's": " when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}


In [ ]:
#Expand words
def decontract(words):
  final = []
  for word in words:
    if word in contractions:
      final.append(contractions[word])
    else:
      final.append(word)
  return " ".join(final)

comments["comment"] = comments["comment"].apply(lambda x: decontract(x.split()))
comments["comment"] = comments["comment"].apply(lambda x: re.sub(r"\'s", " is", x))
comments["parent_comment"] = comments["parent_comment"].apply(lambda x: decontract(x.split()))
comments["parent_comment"] = comments["parent_comment"].apply(lambda x: re.sub(r"\'s", " is", x))
comments.head()

,label,comment,parent_comment,word_count,capital_count,punc_count
0,0,nc and nh.,"yeah, i get that argument. at this point, i wo...",3,4,1
10,0,i think a significant amount would be against ...,i bet if that money was poured into college de...,15,1,1
17,0,because it is what really bothers him... and i...,he actually acts like a moody emo girl on twit...,12,0,6
22,0,conservatism as an ideology is for sure a reac...,"i still doubt that ""all conservatives stand fo...",29,1,2
23,0,"maybe not control, but certainly that is evide...",today russian media tweeted out that wikileaks...,10,1,2


In [ ]:
stopwords

{"'d",
 "'ll",
 "'m",
 "'re",
 "'s",
 "'ve",
 'a',
 'about',
 'above',
 'across',
 'after',
 'afterwards',
 'again',
 'against',
 'all',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'amount',
 'an',
 'and',
 'another',
 'any',
 'anyhow',
 'anyone',
 'anything',
 'anyway',
 'anywhere',
 'are',
 'around',
 'as',
 'at',
 'back',
 'be',
 'became',
 'because',
 'become',
 'becomes',
 'becoming',
 'been',
 'before',
 'beforehand',
 'behind',
 'being',
 'below',
 'beside',
 'besides',
 'between',
 'beyond',
 'both',
 'bottom',
 'but',
 'by',
 'ca',
 'call',
 'can',
 'cannot',
 'could',
 'did',
 'do',
 'does',
 'doing',
 'done',
 'down',
 'due',
 'during',
 'each',
 'eight',
 'either',
 'eleven',
 'else',
 'elsewhere',
 'empty',
 'enough',
 'even',
 'ever',
 'every',
 'everyone',
 'everything',
 'everywhere',
 'except',
 'few',
 'fifteen',
 'fifty',
 'first',
 'five',
 'for',
 'former',
 'formerly',
 'forty',
 'four',
 'from',
 'fron

In [ ]:
#Remove stop words
def remove_stopwords(words):
  final = []
  for word in words:
    if word not in stopwords:
      final.append(word)
  return " ".join(final)

comments["comment"] = comments["comment"].apply(lambda x: remove_stopwords(x.split()))
comments["parent_comment"] = comments["parent_comment"].apply(lambda x: remove_stopwords(x.split()))
comments.head()

,label,comment,parent_comment,word_count,capital_count,punc_count
0,0,nc nh.,"yeah, argument. point, prefer lived nc well.",3,4,1
10,0,think significant spending tax dollars people.,bet money poured college debt health debt reli...,15,1,1
17,0,bothers him... sign weakness.,actually acts like moody emo girl twitter. las...,12,0,6
22,0,conservatism ideology sure reaction liberalism...,"doubt ""all conservatives stand defeating liber...",29,1,2
23,0,"maybe control, certainly evidence collusion.",today russian media tweeted wikileaks released...,10,1,2


In [ ]:
#Removing special characters and punctuation

comments["comment"] = comments["comment"].apply(lambda x: re.sub(r'[^\w ]+', "", x))
comments["parent_comment"] = comments["parent_comment"].apply(lambda x: re.sub(r'[^\w ]+', "", x))
comments

,label,comment,parent_comment,word_count,capital_count,punc_count
0,0,nc nh,yeah argument point prefer lived nc well,3,4,1
10,0,think significant spending tax dollars people,bet money poured college debt health debt reli...,15,1,1
17,0,bothers him sign weakness,actually acts like moody emo girl twitter lash...,12,0,6
22,0,conservatism ideology sure reaction liberalism...,doubt all conservatives stand defeating libera...,29,1,2
23,0,maybe control certainly evidence collusion,today russian media tweeted wikileaks released...,10,1,2
...,...,...,...,...,...,...
1010808,1,women lead men anyway bible,read comment section glenn beck fluffers silen...,9,2,6
1010811,1,thank unions,wonder california broke 350k nurse cali outra...,3,1,1
1010820,1,o,combat troops stay northern iraq deadline pull...,1,1,1
1010824,1,slavs got country called kosovo,unsettled that hear lot jewish people israel p...,11,3,1


In [ ]:
#Frequency of words
text = " ".join(comments["comment"])
parent_text = " ".join(comments["parent_comment"])
text = text.split()
parent_text = parent_text.split()
word_freq = pd.Series(text).value_counts()
parent_freq = pd.Series(parent_text).value_counts()

In [ ]:
#Get top20 words
top_20_words = word_freq[:20]
top_20_words_parent = parent_freq[:20]
top_20_words

,count
people,5371
yeah,4591
like,4572
right,2893
sure,2562
yes,2348
know,2148
trump,2111
good,2101
think,1969


In [ ]:
#get rare words
#rare_words = word_freq.tail(17311) #words that are used once
rare_words = word_freq[word_freq == 1]
rare_words_parent = parent_freq[parent_freq == 1]
rare_words

,count
snack,1
engineermodify,1
dirtraces,1
scrappy,1
ppv,1
...,...
halfbaked,1
itoldyouso,1
kahn,1
nova,1


In [ ]:
#Remove common words and rare words
comments["comment"] = comments["comment"].apply(lambda x: " ".join([t for t in x.split() if t not in rare_words]))
comments["comment"] = comments["comment"].apply(lambda x: " ".join([t for t in x.split() if t not in top_20_words]))
comments["parent_comment"] = comments["parent_comment"].apply(lambda x: " ".join([t for t in x.split() if t not in rare_words_parent]))
comments["parent_comment"] = comments["parent_comment"].apply(lambda x: " ".join([t for t in x.split() if t not in top_20_words_parent]))
comments.head()

,label,comment,parent_comment,word_count,capital_count,punc_count
0,0,nc nh,yeah argument point prefer lived nc well,3,4,1
10,0,significant spending tax dollars,bet poured college debt health debt relief 81 ...,15,1,1
17,0,bothers him sign weakness,actually acts moody girl twitter lashes incred...,12,0,6
22,0,conservatism ideology reaction liberalism deve...,doubt all conservatives stand defeating libera...,29,1,2
23,0,maybe control certainly evidence collusion,today russian media tweeted wikileaks released...,10,1,2


In [ ]:
#Lemmatize
nlp = spacy.load("en_core_web_sm")
def make_to_base(x):
  x = str(x)
  x_list = []
  doc = nlp(x)
  for token in doc:
    lemma = token.lemma_
    if lemma == "-PRON-" or lemma == "be":
      lemma = token.text
    x_list.append(lemma)
  return " ".join(x_list)

comments["comment"] = comments["comment"].apply(lambda x: make_to_base(x))
comments["parent_comment"] = comments["parent_comment"].apply(lambda x: make_to_base(x))
comments.head()




,label,comment,parent_comment,word_count,capital_count,punc_count
0,0,nc nh,yeah argument point prefer live nc well,3,4,1
10,0,significant spending tax dollar,bet pour college debt health debt relief 81 am...,15,1,1
17,0,bother he sign weakness,actually act moody girl twitter lash incredibl...,12,0,6
22,0,conservatism ideology reaction liberalism deve...,doubt all conservative stand defeat liberal us...,29,1,2
23,0,maybe control certainly evidence collusion,today russian medium tweet wikileak release po...,10,1,2


In [ ]:
from textblob import TextBlob
from textblob.sentiments import PatternAnalyzer

def get_polarity(x):
  return TextBlob(x, analyzer=PatternAnalyzer()).sentiment.polarity

def get_subjectivity(x):
  return TextBlob(x, analyzer=PatternAnalyzer()).sentiment.subjectivity

comments["comment_polarity"] = comments["comment"].apply(lambda x: get_polarity(x))
comments["comment_subjectivity"] = comments["comment"].apply(lambda x: get_subjectivity(x))
comments["parent_polarity"] = comments["parent_comment"].apply(lambda x: get_polarity(x))
comments["parent_subjectivity"] = comments["parent_comment"].apply(lambda x: get_subjectivity(x))
comments.sample(5)

,label,comment,parent_comment,word_count,capital_count,punc_count,comment_polarity,comment_subjectivity,parent_polarity,parent_subjectivity
198940,1,mistake hear,little c guy,11,1,1,0.00,0.0,-0.187500,0.50
909485,0,month repeat this no goal trope everywhere,tangible goal politic eliminate problem relate...,13,1,6,0.00,0.0,0.000000,0.00
958458,0,narrow view come natural,refuse job instead leech welfare starve now ge...,11,1,1,-0.05,0.4,0.468182,0.75
237623,1,mentally ill white guy,soooo neonazi lunatic islamic extremist hurry ...,5,1,1,-0.25,0.5,0.000000,0.00
483964,1,ok,dishonest lie writer btfo be hope career that,7,2,3,0.50,0.5,-0.300000,0.50


In [ ]:
#Export
from google.colab import drive
drive.mount('/content/drive')
comments.to_csv('/content/drive/My Drive/sentiment.csv', index=False)

Mounted at /content/drive


NameError: name 'comments' is not defined

In [ ]:
#Verify
sentiment = pd.read_csv('/content/drive/My Drive/sentiment.csv')
sentiment.sample(5)

,label,comment,parent_comment,word_count,capital_count,punc_count,comment_polarity,comment_subjectivity,parent_polarity,parent_subjectivity
37502,1,fortunately get little shindig,gop spend investigate bill blowjob,10,1,1,0.10625,0.6,0.00,0.00
77614,1,canadian oil nicer,think oil spill all,7,2,3,0.00000,0.0,0.00,0.00
32761,1,fair,low caste indian girl 11 face shun resist rape...,2,1,2,0.70000,0.9,0.08,0.42
82071,0,hear context,ron paul an education medical care,4,1,1,0.00000,0.0,0.00,0.00
70693,0,socialism society capitalism big pile,welcome capitalism,12,3,6,0.00000,0.1,0.80,0.90


In [ ]:
#Auto correct
# spell = Speller(lang="en")
# comments["comment"] = comments["comment"].apply(lambda x: spell(x))
# comments.head()

In [ ]:
#TfidfVectorizer
# from sklearn.feature_extraction.text import TfidfVectorizer

# v = TfidfVectorizer()
# corpus = comments["comment"]
# transformed = v.fit_transform(corpus)
# transformed.shape